<a href="https://colab.research.google.com/github/chris051091/Seminario-3/blob/master/Analysis/Eventos_Estaciones/Analisis_Eventos_Palanquero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import io
from datetime import timedelta

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
PATH_DRIVE = 'drive/MyDrive/Seminario-3-master'
PATH_DADOS_EVENTOS = '/dados/'
FILE_DATOS_ESTACION = 'eventos_palanquero.csv'

PATH_DADOS_PREPROCESS = '/preprocess/dados/palanquero/tratados/'
FILE_TEMP = 'palanquero_tratados_temp.csv'
FILE_HUM = 'palanquero_tratados_hum.csv'
FILE_PRES = 'palanquero_tratados_pres.csv'
FILE_PREC = 'palanquero_tratados_prec.csv'
FILE_VIENTO = 'palanquero_tratados_viento.csv'

# **Cargando eventos**

In [38]:
dfEventos = pd.read_csv(PATH_DRIVE + PATH_DADOS_EVENTOS + FILE_DATOS_ESTACION, 
                 sep = ';'
                )
dfEventos = dfEventos.sort_values(by='date_min_t').reset_index(drop = True)
dfEventos['FECHA'] = pd.to_datetime(dfEventos['date_min_t'], format='%Y %m %d %H %M').dt.tz_localize('America/Bogota')
dfEventos['FECHA_MIN'] = dfEventos['FECHA'].dt.round('10min')  
dfEventos['FECHA'] = dfEventos['FECHA'].dt.round('H')
dfEventos = dfEventos[['FECHA', 'FECHA_MIN', 'min_t', 'lon_t', 'lat_t']]
dfEventos

,FECHA,FECHA_MIN,min_t,lon_t,lat_t
0,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,198.5,-74.693,5.484
1,2011-11-03 02:00:00-05:00,2011-11-03 02:20:00-05:00,191.9,-74.693,5.484
2,2011-12-11 03:00:00-05:00,2011-12-11 02:40:00-05:00,199.3,-74.693,5.484
3,2012-03-16 23:00:00-05:00,2012-03-16 22:40:00-05:00,193.6,-74.693,5.484
4,2012-09-11 05:00:00-05:00,2012-09-11 04:40:00-05:00,217.9,-74.693,5.484
5,2013-07-05 01:00:00-05:00,2013-07-05 01:20:00-05:00,226.0,-74.693,5.484
6,2013-11-11 00:00:00-05:00,2013-11-11 00:20:00-05:00,212.0,-74.693,5.484
7,2013-11-11 01:00:00-05:00,2013-11-11 00:40:00-05:00,208.5,-74.693,5.484
8,2014-10-19 01:00:00-05:00,2014-10-19 00:40:00-05:00,196.3,-74.693,5.484
9,2014-12-08 00:00:00-05:00,2014-12-07 23:40:00-05:00,219.2,-74.693,5.484


In [7]:
FILES = [FILE_PRES, FILE_HUM, FILE_TEMP]
df = dfEventos.copy()

for FILE in FILES :
  
  dfVar = pd.read_csv(PATH_DRIVE + PATH_DADOS_PREPROCESS + FILE, 
                 sep = ';', 
                 parse_dates = ["FECHA"],
                 date_parser = lambda col: pd.to_datetime(col)
                )
  
  df = pd.merge(df, dfVar, on=['FECHA'], how = 'left')
  df['FECHA'] = pd.to_datetime(df["FECHA"])

In [8]:
VARS = ['TEMP', 'PRESION', 'HUM']

for VAR in VARS:
  df[f'{VAR}_MEAN'] = df[VAR].mean()

df

,FECHA,FECHA_MIN,min_t,lon_t,lat_t,PRESION,HUM,TEMP,TEMP_MEAN,PRESION_MEAN,HUM_MEAN
0,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,198.5,-74.693,5.484,NaN,NaN,NaN,24.64,989.86,87.333333
1,2011-11-03 02:00:00-05:00,2011-11-03 02:20:00-05:00,191.9,-74.693,5.484,988.4,NaN,25.6,24.64,989.86,87.333333
2,2011-12-11 03:00:00-05:00,2011-12-11 02:40:00-05:00,199.3,-74.693,5.484,NaN,NaN,NaN,24.64,989.86,87.333333
3,2012-03-16 23:00:00-05:00,2012-03-16 22:40:00-05:00,193.6,-74.693,5.484,NaN,NaN,NaN,24.64,989.86,87.333333
4,2012-09-11 05:00:00-05:00,2012-09-11 04:40:00-05:00,217.9,-74.693,5.484,990.2,80.0,24.6,24.64,989.86,87.333333
5,2013-07-05 01:00:00-05:00,2013-07-05 01:20:00-05:00,226.0,-74.693,5.484,991.3,98.0,24.9,24.64,989.86,87.333333
6,2013-11-11 00:00:00-05:00,2013-11-11 00:20:00-05:00,212.0,-74.693,5.484,990.2,NaN,25.0,24.64,989.86,87.333333
7,2013-11-11 01:00:00-05:00,2013-11-11 00:40:00-05:00,208.5,-74.693,5.484,990.2,NaN,23.4,24.64,989.86,87.333333
8,2014-10-19 01:00:00-05:00,2014-10-19 00:40:00-05:00,196.3,-74.693,5.484,993.7,100.0,22.2,24.64,989.86,87.333333
9,2014-12-08 00:00:00-05:00,2014-12-07 23:40:00-05:00,219.2,-74.693,5.484,989.6,98.0,24.9,24.64,989.86,87.333333


In [56]:
FILES = [FILE_PREC, FILE_VIENTO]

df10Min = dfEventos.copy()
for FILE in FILES :
  
  dfVar = (pd.read_csv(PATH_DRIVE + PATH_DADOS_PREPROCESS + FILE, 
                 sep = ';', 
                 parse_dates = ["FECHA"],
                 date_parser = lambda col: pd.to_datetime(col)
                ).rename(columns = {'FECHA' : 'FECHA_MIN'})
          )
  
  df10Min = pd.merge(df10Min, dfVar, on=['FECHA_MIN'], how = 'left')
  df10Min['FECHA_MIN'] = pd.to_datetime(df["FECHA_MIN"])

In [57]:
df10Min

,FECHA,FECHA_MIN,min_t,lon_t,lat_t,PREC,DIRECVIENTO,VELVIENTO
0,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,198.5,-74.693,5.484,NaN,NaN,NaN
1,2011-11-03 02:00:00-05:00,2011-11-03 02:20:00-05:00,191.9,-74.693,5.484,0.0,NaN,NaN
2,2011-12-11 03:00:00-05:00,2011-12-11 02:40:00-05:00,199.3,-74.693,5.484,NaN,NaN,NaN
3,2012-03-16 23:00:00-05:00,2012-03-16 22:40:00-05:00,193.6,-74.693,5.484,NaN,NaN,NaN
4,2012-09-11 05:00:00-05:00,2012-09-11 04:40:00-05:00,217.9,-74.693,5.484,0.0,NaN,NaN
5,2013-07-05 01:00:00-05:00,2013-07-05 01:20:00-05:00,226.0,-74.693,5.484,0.0,56.0,0.2
6,2013-11-11 00:00:00-05:00,2013-11-11 00:20:00-05:00,212.0,-74.693,5.484,16.7,251.0,0.5
7,2013-11-11 01:00:00-05:00,2013-11-11 00:40:00-05:00,208.5,-74.693,5.484,6.9,104.0,0.3
8,2014-10-19 01:00:00-05:00,2014-10-19 00:40:00-05:00,196.3,-74.693,5.484,1.0,98.0,0.2
9,2014-12-08 00:00:00-05:00,2014-12-07 23:40:00-05:00,219.2,-74.693,5.484,0.0,30.0,0.7


In [48]:
funtime = lambda x, hour : [x - timedelta(hours = h) for h in (range(0, hour+1) if hour > 0 else range(hour, 0))]

# [base - datetime.timedelta(days=x) for x in range(numdays)]
dfEventAn = dfEventos.copy()
pd.set_option('display.max_colwidth', None)
dfEventAn['FECHA_ANTES'] = dfEventAn['FECHA'].apply(funtime, hour = 6)
dfEventAn['FECHA_DESPUES'] = dfEventAn['FECHA'].apply(funtime, hour = -6)
dfEventAn['FECHA_INTERVALO'] = dfEventAn['FECHA_ANTES'] + dfEventAn['FECHA_DESPUES']
dfEventAn = dfEventAn.drop(columns=['FECHA_ANTES', 'FECHA_DESPUES']).explode('FECHA_INTERVALO')
dfEventAn['FECHA_POS'] = (dfEventAn['FECHA_INTERVALO'] - dfEventAn['FECHA'])/ pd.Timedelta(hours=1)
dfEventAn = dfEventAn.rename(columns={'FECHA' : 'FECHA_EVENTO', 'FECHA_INTERVALO': 'FECHA'})


dfEventAn.head(15)

,FECHA_EVENTO,FECHA_MIN,min_t,lon_t,lat_t,FECHA,FECHA_POS
0,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,198.5,-74.693,5.484,2011-06-06 22:00:00-05:00,0.0
0,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,198.5,-74.693,5.484,2011-06-06 21:00:00-05:00,-1.0
0,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,198.5,-74.693,5.484,2011-06-06 20:00:00-05:00,-2.0
0,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,198.5,-74.693,5.484,2011-06-06 19:00:00-05:00,-3.0
0,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,198.5,-74.693,5.484,2011-06-06 18:00:00-05:00,-4.0
0,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,198.5,-74.693,5.484,2011-06-06 17:00:00-05:00,-5.0
0,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,198.5,-74.693,5.484,2011-06-06 16:00:00-05:00,-6.0
0,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,198.5,-74.693,5.484,2011-06-07 04:00:00-05:00,6.0
0,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,198.5,-74.693,5.484,2011-06-07 03:00:00-05:00,5.0
0,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,198.5,-74.693,5.484,2011-06-07 02:00:00-05:00,4.0


In [66]:
FILES = [FILE_PRES, FILE_HUM, FILE_TEMP]
dfCross = dfEventAn.copy()

for FILE in FILES :
  
  dfVar = pd.read_csv(PATH_DRIVE + PATH_DADOS_PREPROCESS + FILE, 
                 sep = ';', 
                 parse_dates = ["FECHA"],
                 date_parser = lambda col: pd.to_datetime(col)
                )
  
  dfCross = pd.merge(dfCross, dfVar, on=['FECHA'], how = 'left')
  dfCross['FECHA'] = pd.to_datetime(dfCross["FECHA"])

VARS = ['TEMP', 'PRESION', 'HUM']
dfVars = pd.DataFrame()

for VAR in VARS:
  dfVars[f'{VAR}_MEAN'] = dfCross.groupby('FECHA_POS')[VAR].mean()

dfVars = dfVars.reset_index()

## Estamos colocando los promedios de los 16 eventos, considerando 6 horas antes y después
dfVars.head(15)

,FECHA_POS,TEMP_MEAN,PRESION_MEAN,HUM_MEAN
0,-6.0,28.458333,987.241667,78.125000
1,-5.0,28.050000,987.766667,79.250000
2,-4.0,27.875000,988.118182,78.875000
3,-3.0,26.963636,988.936364,85.142857
4,-2.0,25.790000,989.309091,89.285714
5,-1.0,25.600000,989.500000,90.333333
6,0.0,24.640000,989.860000,87.333333
7,1.0,23.690000,989.900000,86.666667
8,2.0,23.780000,989.680000,86.666667
9,3.0,24.044444,989.770000,85.833333
